In [1]:
import string
import sys

import numpy as np
import pandas as pd

sys.path.append("..")
from heat.model import HeatModel
from NGS.data import NGSDataset, add_missing, add_noise, preprocess
from NGS.experiment import run
from NGS.hyperparameter import HyperParameter
from path import DATA_DIR

In [2]:
# Random experiment ID
exp_id = "".join(np.random.choice(list(string.ascii_lowercase + string.digits), 8))

hp = HyperParameter(
    dataset="heat_train",
    seed=42,
    missing=0.1,
    noise=0.001,
    depth=2,
    emb_dim=8,
    threshold=None,
    device="cuda:0",
    epochs=1540,
    batch_size=16,
)

In [3]:
df = pd.read_pickle(DATA_DIR / f"{hp.dataset}.pkl")
train, val = preprocess(df, val_ratio=0.2)

# Incomplete data
rng = np.random.default_rng(hp.seed)
seed_missing, seed_noise = rng.integers(42, size=(2,))
add_missing(train, hp.missing, seed_missing)
add_noise(train, hp.noise, seed_noise)

# Dataset for train, validation, rollout validation
train_dataset = NGSDataset(**train, window=1)
val_dataset = NGSDataset(**val, window=1)
rollout_dataset = NGSDataset(**val, window=-1)

# Model
model = HeatModel(hp.emb_dim, hp.depth)

In [4]:
print(f"Start running {exp_id=}")
run(exp_id, hp, model, train_dataset, val_dataset, rollout_dataset)

Start running exp_id='temp'
Best at epoch=0, train_mae=1.6186e-02, val_mae=1.6772e-02, rollout_mae=2.4842e-01
Best at epoch=1, train_mae=1.1798e-02, val_mae=1.1159e-02, rollout_mae=6.4831e-02


FileExistsError: [Errno 17] File exists: '/pds/pds11/hoyun/NGS/NGS_public/result/temp'